In [1]:
import numpy as np

### Instantiate the prediction task

In [2]:
from jenga.tasks.income import IncomeEstimationTask

task = IncomeEstimationTask(seed=42)

### Task details

In this task, we try to predict the income level of a person (more or less than 50K dollars per years) from demographic and work-related data. It is often used as a proxy to study automated decision making for loan applications.

The original data is available in the UCI machine learning repository at https://archive.ics.uci.edu/ml/datasets/adult

In [3]:
task.train_data

,workclass,occupation,marital_status,education,hours_per_week,age
25373,Self-emp-inc,Craft-repair,Never-married,Bachelors,20,38
8853,NaN,NaN,Widowed,HS-grad,12,68
21978,Local-gov,Prof-specialty,Married-civ-spouse,Bachelors,35,41
6923,Private,Tech-support,Married-civ-spouse,Some-college,45,30
14401,Self-emp-not-inc,Exec-managerial,Divorced,HS-grad,70,37
...,...,...,...,...,...,...
4986,Federal-gov,Other-service,Never-married,12th,30,19
10316,State-gov,Exec-managerial,Never-married,Masters,35,40
4348,Private,Handlers-cleaners,Divorced,Some-college,84,40
9115,Private,Exec-managerial,Divorced,Bachelors,40,38


### Train the provided baseline model

Jenga allows us to easily train and evaluate a logistic regression model for this task. Have a look at https://github.com/schelterlabs/jenga/blob/master/jenga/tasks/income.py if you want to know the details.


In [4]:
model = task.fit_baseline_model()

In [5]:
f"The ROC AUC score on the test data is {task.get_baseline_performance()}"

'The ROC AUC score on the test data is 0.8731939181182589'

### Let's have a look at the test data

In [6]:
task.test_data

,workclass,occupation,marital_status,education,hours_per_week,age
30835,Private,Craft-repair,Married-civ-spouse,Assoc-voc,38,45
23857,State-gov,Protective-serv,Married-civ-spouse,Some-college,40,42
251,Private,Adm-clerical,Married-civ-spouse,Bachelors,40,42
20208,Self-emp-inc,Sales,Married-civ-spouse,Bachelors,65,33
15120,State-gov,Adm-clerical,Married-civ-spouse,Some-college,38,31
...,...,...,...,...,...,...
22279,Self-emp-inc,Farming-fishing,Married-civ-spouse,Some-college,40,40
4736,Private,Exec-managerial,Married-civ-spouse,HS-grad,84,25
8083,Private,Other-service,Married-civ-spouse,Some-college,40,40
11862,Private,Handlers-cleaners,Never-married,HS-grad,40,23


### Corruptions

Jenga provides a set of predefined data corruptions that we can use to simulate errors in the test data. We will simulate 'implicit missing values' in a column and see how this impacts the prediction quality.

Jenga supports much more predefined corruptions, have a look at https://github.com/schelterlabs/jenga/tree/master/jenga/corruptions.


In [7]:
from jenga.corruptions.generic import MissingValues

marital_status_corruption = MissingValues(column='marital_status', fraction=0.99, na_value='BROKEN')

In [8]:
corrupted_test_data = marital_status_corruption.transform(task.test_data)
corrupted_test_data

,workclass,occupation,marital_status,education,hours_per_week,age
30835,Private,Craft-repair,BROKEN,Assoc-voc,38,45
23857,State-gov,Protective-serv,BROKEN,Some-college,40,42
251,Private,Adm-clerical,BROKEN,Bachelors,40,42
20208,Self-emp-inc,Sales,BROKEN,Bachelors,65,33
15120,State-gov,Adm-clerical,BROKEN,Some-college,38,31
...,...,...,...,...,...,...
22279,Self-emp-inc,Farming-fishing,BROKEN,Some-college,40,40
4736,Private,Exec-managerial,BROKEN,HS-grad,84,25
8083,Private,Other-service,BROKEN,Some-college,40,40
11862,Private,Handlers-cleaners,BROKEN,HS-grad,40,23


In [9]:
y_pred = model.predict_proba(corrupted_test_data)

f"The ROC AUC score on the corrupted test data is {task.score_on_test_data(y_pred)}"

'The ROC AUC score on the corrupted test data is 0.8000869740922627'

### Jenga's evaluators

Jenga provides a set of evaluators which allow us to automate the evaluation of the impact of given data corruptions.

In [10]:
from jenga.evaluation.corruption_impact import CorruptionImpactEvaluator

evaluator = CorruptionImpactEvaluator(task)

In [11]:
corruptions = [
    MissingValues(column='marital_status', fraction=0.99, na_value='BROKEN'),
    MissingValues(column='age', fraction=0.05, na_value=-999),
]

### Run the evaluation of the corruptions with 10 repetitions 

In [12]:
num_repetitions = 10
results = evaluator.evaluate(model, num_repetitions, *corruptions)

0/20 (0.019917999999999658)
10/20 (0.21423999999999932)


### Investigate the impact on the predictive performance of the model

In [13]:
for validation_result in results:
    
    print(validation_result.corruption)
    print(f"""
     Score (AUC) on 
      clean data:     {validation_result.baseline_score}
      corrupted data: {np.mean(validation_result.corrupted_scores)}
     """)    
    print("\n")

MissingValues: {'column': 'marital_status', 'fraction': 0.99, 'sampling': 'MCAR', 'na_value': 'BROKEN'}

     Score (AUC) on 
      clean data:     0.8731939181182589
      corrupted data: 0.7999927909174862
     


MissingValues: {'column': 'age', 'fraction': 0.05, 'sampling': 'MCAR', 'na_value': -999}

     Score (AUC) on 
      clean data:     0.8731939181182589
      corrupted data: 0.8366003181298348
     


